# Turing Machines

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try|:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True
    
if OWN_INSTALL:
    
  #---- Leave these definitions ON if running on laptop
  #---- Else turn OFF by putting them between ''' ... '''

  sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',  
                   '../../../..',  '../../../../3rdparty',  
                   '../../..',     '../../../3rdparty', 
                   '../..',        '../../3rdparty',
                   '..',           '../3rdparty' ]

else: # In colab
  ! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
  sys.path.append('./Jove')
  sys.path.append('./Jove/jove')

# -- common imports --
from jove.Def_TM     import *
from jove.Def_md2mc  import *
from jove.DotBashers import *
from jove.JoveEditor import *
from jove.Def_PDA    import *
from jove.AnimateTM import * #-- usage: AnimateTM(whas101DTM, FuseEdges=True)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

## Question 1 (25 points):

### Solve the PCP instance given below by hand first (15 points) 


### TILES = [('110', '1'), ('1', '0'), ('0', '110'),       ('0','1')] 

### And then by using the tool (10 points)

### using the command pcp_solve( TILES  , OWN_INSTALL)  

### You may always use help(pcp_solve) for details

In [ ]:
Solution 1
[3, 2, 2, 4, 1]
0   1   1   0   011 
011 0   0   1   1  

# Halt_TM proof
 
 
* Suppose there exists a decider $H$ for $Halt_{TM}$. 
 $H$ expects to be
 given a \TM $M$ and a string $w$. Notice that _giving a TM to $H$_
 means
 ``giving it a character string representing a TM program.''
 Hence, in reality, we will be feeding $H$
  the pair $\langle M,w\rangle$.

* Build a program called $D$ as follows: 
  - $D$ takes a single argument $M$.
  - As its first step, $D$ invokes $H$ on $\langle M,M\rangle$.
  - If $H(\langle M,M\rangle)$ rejects, $D(\langle M\rangle)$ accepts (hence halts).
  - If $H(\langle M,M\rangle)$ accepts, $D(\langle M\rangle)$ loops.
    
 
* Now we can ask what $D(\langle D\rangle)$ will 
  - The $D(\langle D\rangle)$ ``call'' turns into an $H(\langle D,D\rangle)$ 
    call.
 
* Suppose $H(\langle D,D\rangle)$ rejects. then $D(\langle D\rangle)$ halts.

* But, according to the advertised behavior of $H$, the
   fact that $H(\langle D,D\rangle)$
   rejects means that $D$ is 
   a Turing machine that {\bf does not} halt on $\langle D\rangle$, or
  that $D(\langle D\rangle)$   loops!  
 
* Suppose $H(\langle D,D\rangle)$ accepts. 
  In that case,  
  $D(\langle D\rangle)$ loops.

* But, according to the advertised behavior of $H$, the
  fact that $H(\langle D,D\rangle)$
  accepts means that $D$ {\em is} 
   a Turing machine that {\em halts on} $\langle D\rangle$, or
   that 
   $D(\langle D\rangle)$ halts!}   This is
  the second part of a two-part contradiction that we
  have obtained.


* Therefore, we obtain a contradiction under
  all cases. 
  
* Thus, $Halt_{TM}$ is undecidable.

 
 

## CFL_TM Mapping Reduction


* The mapping reduction from $A_{TM}$ to $CFL_{TM}$ is to emit this machine M' 
 

      M'(x) {
        if x is of the form 0^n 1^n 2^n then goto accept_M' ;  // Could also be a "ww" pattern which is non-CFL
        Run M on w ;
        If M accepts w, goto accept_M' ;
        If M rejects w, goto reject_M' ; }
 

* Suppose M' has a CFL
  - but this is possible only if the language of M' is 0^n 1^n 2^n UNION all remaining strings, i.e. Sigma*
  - This is possible if M accepts w
  
* Suppose M' does not have a CFL
  - this is possible only if  M does not accept w (in that case, strings 0^n 1^n 2^n will be in M's language)
  
* Thus we have achieved our mapping reduction 
  - that is, M' has a CFL if and only if M accepts w.

# Asg9 DTM for w#x where w is a substring of x

In [ ]:
Asg9DTM = md2mc('''
TM 
I : #; #, R  -> F       !! Done since no w or x
I : a; A, R -> LCa      !! found a; change to mark as read (A), seek # left of #, then a
I : b; B, R -> LCb      !! ditto except a->b

LCa: a; a, R | b; b, R -> LCa !! keep skipping seek #
LCb: a; a, R | b; b, R -> LCb !! ditto - seek #

LCa: #; #, R -> RCa  !! Seek a to right of #
LCb: #; #, R -> RCb  !! ditto, but b

RCa: X; X, R | Y; Y, R -> RCa !! Keep looking for a 
RCb: X; X, R | Y; Y, R -> RCb !! .. for b

RCa: a; X, L -> MoreL#  !! found first a; enter more loop, but first get #
RCb: b; Y, L -> MoreL#  !! found first b; enter more loop, but first get #

RCa: b; Y, L -> ClrI !! Failed finding a; found b instead; enter clear loop
RCb: a; X, L -> ClrI !! ditto : a->b

ClrI: X; X, L | Y; Y, L | #; #, L | a; a, L | b; b, L | A; a, L | B; b, L -> ClrI !! clear loop

ClrI: .; ., R -> I !! Clear loop over, back to pristine "I" state


MoreL#: X; X, L | Y; Y, L | a; a, L | b; b, L | A; A, L | B; B, L -> MoreL# !! More loop, first get #

MoreL#: #; #, L -> MoreL !! More loop; Found #; now plain more Loop

MoreL:  a; a, L | b; b, L -> MoreL !! More loop for left end

MoreL: A; A, R -> J !! Successful More loop fini ; next iteration begins at J (not I)
MoreL: B; B, R -> J !! ditto (A->B)

J: #; #, R -> F !! at J, if see #, no more; success

J: a; A, R -> LCa2  !! J loop imitates I
J: b; B, R -> LCb2  !! ditto a->b

LCa2: a; a, R | b; b, R -> LCa2 !! In J loop, LCa2 imitates LCa
LCb2: a; a, R | b; b, R -> LCb2 !! ditto a->b

LCa2: #; #, R -> RCa2 !! In J loop, RCa2 imitates RCa
LCb2: #; #, R -> RCb2 !! ditto a->b

RCa2: X; X, R | Y; Y, R | A; A, R | B; B, R -> RCa2 !! Imitation of RCa
RCb2: X; X, R | Y; Y, R | A; A, R | B; B, R -> RCb2 !! Imitation of RCb

RCa2: a; A, L -> MoreL# !! Imitation of RCa
RCb2: b; B, L -> MoreL# !! Imitation of RCb

RCa2: b; b, L -> ClrI !! Imitation of RCa
RCb2: a; a, L -> ClrI !! Imitation of RCb
 
RCa: .; ., R -> Stuck !! If x shorter, stuck
RCb: .; ., R -> Stuck !! ditto
''')


# Tests for the DTM

### (a) aba#aba – pass 
### (b) aba#ab – fail
### (c) aba#baba – pass 
### (d) aba#babba –fail
### (e) bba#bbabba – pass


## Please include five more tests of your own

In [ ]:
help(explore_tm)

In [ ]:
#Pass
explore_tm(Asg9DTM, "aba#aba",50)

In [ ]:
#Fail
explore_tm(Asg9DTM, "aba#ab",500)

In [ ]:
#Pass
explore_tm(Asg9DTM, "aba#baba",50)

In [ ]:
#Fail
explore_tm(Asg9DTM, "aba#babba",500)

In [ ]:
#Pass
explore_tm(Asg9DTM, "bba#bbabba",50)

In [ ]:
# Test submitted by student - PASS
explore_tm(Asg9DTM, "ababba#bbabababba",296)

In [ ]:
#Fail
explore_tm(Asg9DTM, "ababba#bbabababa",596)

In [ ]:
#Pass
explore_tm(Asg9DTM, "bbabababa#bbabababa",596)

In [ ]:
#Pass
explore_tm(Asg9DTM, "bbabababa#ababababbbabababa",1096)

In [ ]:
#Pass
explore_tm(Asg9DTM, "bbabababa#abbbbabababaabababbbabababa",1096)

# Asg9 NDTM for w#x where w is a substring of x

In [ ]:
# w#x NDTM
#
# An NDTM in which we need to locate whether w is a substring of x
# Once a match is found at an arbitrary location, we accept
#
#
'''
The alphabet is {a,b}

The steps are as follows.

1) Locate the #

2) Then go past the # and locate some cell and capitalize it
  a) if it is a a, change it to an A
  b) if it is a b, change it to a B
 
3) Then sweep back and compare w cell by cell with what's beyond #
    beginning with the capitalized letter.
   
4) We could use some cleverness in that the first capitalization
   itself could be for a possible match. Else it is a failure!
   This may not pay off and bloat the code 2x
   
So the pseudo-code is as follows

init:
 Simply go seek an ND location after #,
 change it to '1' to mark it as 'a'
 change it to '2' to mark it as 'b'
 Then return to compare w with where things are 1 or 2
 change back to A or B and compare then on

'''

Asg9NDTM = md2mc('''TM
I : #; #,S -> F
I : a; a,R | b; b,R -> Sk#

Sk# : a; a,R | b; b, R -> Sk#
Sk# : #; #,R -> Mark1

Mark1: a; 1,L | b; 2,L -> LSk#
Mark1: a; a,R | b; b,R -> Mark1
Mark1: .; .,S -> BH !! ran off end; that ND path can crash too

LSk# : a; a,L | b; b,L | 1; 1,L | 2; 2,L | A; A,L | B; B,L -> LSk#
LSk# : #; #,L -> Sklend

Sklend: a; a,L | b; b,L -> Sklend
Sklend: .; .,R | A; A,R | B; B, R -> Comp

Comp: A; A,R | B; B,R -> Comp

Comp : a; A,R -> Mtcha
Comp : b; B,R -> Mtchb

Comp : #; #,S -> Fini

Mtcha : a; a,R | b; b,R -> Mtcha
Mtchb : a; a,R | b; b,R -> Mtchb

Mtcha : #; #,R -> Nowa
Mtchb : #; #,R -> Nowb

Nowa : a; a,R | b; b,R -> Nowa
Nowa : A; A,R | B; B,R -> NowRa

Nowb : a; a,R | b; b,R -> Nowb
Nowb : A; A,R | B; B,R -> NowRb

Nowa : 1; A,L  -> LSk#
Nowb : 2; B,L  -> LSk#

NowRa : A; A,R | B; B,R -> NowRa
NowRb : A; A,R | B; B,R -> NowRb

NowRa : a; A,L -> LSk#
NowRb : b; B,L -> LSk#

''')



In [ ]:
explore_tm(Asg9NDTM, "aba#aba",50)

In [ ]:
explore_tm(Asg9NDTM, "aba#ab",50)

In [ ]:
explore_tm(Asg9NDTM, "aba#baba",50)

In [ ]:
explore_tm(Asg9NDTM, "aba#babba",50)

In [ ]:
explore_tm(Asg9NDTM, "bba#bbabba",50)

# END